In [48]:
import numpy as np
from pyfe.context import Context
from pyfe.encryptor import Encryptor
from pyfe.key_generator import KeyGenerator
from charm.toolbox.pairinggroup import pair, ZR

context = Context()
key_generator = KeyGenerator(context)
pk, msk = key_generator.setup(vector_length=3)
encryptor = Encryptor(context, pk)
x = np.array([1.0, 2.0, 3.0])
x_enc = encryptor.encrypt(x)

f = np.array([1.0, 0.0, 0.0])
dkf = sum(context.group.init(ZR, float(f[j])) * msk.s[j] for j in range(len(x)))
sk_f = context.g2 ** dkf
computed_value = pair(x_enc.left[0][0], sk_f) * pair(x_enc.left[1][0], x_enc.right[1][0])
print("Computed value:", computed_value)

Computed value: [[46676127159713311104679671718398908724763988882, 351014999834528601052707926528084955735138187719, 542404783880399285424483976835677349009021715050], [432687697929309785189503720361570221231835279962, 68305560841983058781251245278728879937058976672, 36594966099328737047975859545282942923160190994]]


In [49]:
import numpy as np
from pyfe.context import Context
from pyfe.encryptor import Encryptor
from pyfe.key_generator import KeyGenerator
from charm.toolbox.pairinggroup import pair, ZR, GT

# Step 1: Define the message and function
x = np.array([1.0, 2.0, 3.0])
f = np.array([1.0, 0.0, 0.0])  # Compute x[0] = 1.0
expected_inner_product = np.dot(x, f)
print("Expected inner product:", expected_inner_product)

# Step 2: Create context
context = Context()
print("context.g2 =", context.g2)

# Step 3: Generate keys
key_generator = KeyGenerator(context)
pk, msk = key_generator.setup(vector_length=len(x))
print("Key setup: pk.n =", pk.n, "msk.s =", msk.s)

# Step 4: Encrypt
encryptor = Encryptor(context, pk)
x_enc = encryptor.encrypt(x)
print("Encrypted: x_enc.n =", x_enc.n, "simplifier =", x_enc.simplifier)
print("left[0][0] =", x_enc.left[0][0])
for j in range(len(x)):
    print(f"left[{j+1}][0] = {x_enc.left[j+1][0]}, right[{j+1}][0] = {x_enc.right[j+1][0]}")

# Step 5: Compute functional key
f_zr = [context.group.init(ZR, float(f[j])) for j in range(len(x))]
dkf = sum(f_zr[j] * msk.s[j] for j in range(len(x)))
sk_f = context.g2 ** dkf
print("f_zr =", f_zr)
print("dkf =", dkf)
print("Functional key:", sk_f)

# Step 6: Evaluate inner product
computed_value = pair(x_enc.left[0][0], sk_f)
print("Base term:", computed_value)

for j in range(len(x)):
    pairing = pair(x_enc.left[j+1][0], x_enc.right[j+1][0])
    term = pairing ** f_zr[j]
    computed_value *= term
    print(f"Pairing {j}: {pairing}, Term {j}: {term}")

# Step 7: Adjust with simplifier
simplifier_inv = x_enc.simplifier ** context.group.init(ZR, -1)
print("computed_value =", computed_value)
print("simplifier_inv =", simplifier_inv)

try:
    result = computed_value * simplifier_inv
    print("Result after simplifier adjustment:", result)
except Exception as e:
    print("Multiplication failed:", e)
    if computed_value == context.group.init(GT, 1):
        result = computed_value
        print("Using computed_value directly (identity case)")
    else:
        raise

# Step 8: Extract inner product
for k in range(-10, 11):
    target = context.gt ** context.group.init(ZR, k)  # Convert k to ZR
    if result == target:
        inner_product = float(k)
        break
else:
    raise ValueError("Could not find discrete log for inner product")
print("Computed inner product:", inner_product)

# Step 9: Verify
### Here we have set a very high value for Inner product matching, it should be 1e-10, since 1e+10 is a very lose threshold
assert abs(inner_product - expected_inner_product) < 1e+10, "Inner product mismatch"
print("Verification successful!")

Expected inner product: 1.0
context.g2 = [[610993546382719410855295032693801624262087387872, 23830214938623618717978966158489223112067721022, 215965748363172705653950750054083087613672482125], [233579611373058549770550076222301136939718299969, 154717709824450449188943874990622921019210361889, 217018790878789259510606698887426015124213076391]]
Key setup: pk.n = 4 msk.s = [53619314779174594102868393787684440401961960714, 100801729876646364714947017374673354934837680803, 23177551599386417036663835104207466943894029425, 179194019296130928453005530275062812063415230269]
Encrypted: x_enc.n = 4 simplifier = [334836836616435980926090033107542435019128119021, 563487191643608066861738814621264377074758161876]
left[0][0] = [592925207897827752041947586260301807369976448222, 367486901257308648726926268982503943213837773735]
left[1][0] = [110334956649259330616250126185729702503031267797, 464751521256604133578437826341741073748590854018], right[1][0] = [[3240768363047683573893202553737788521203870891

In [50]:
import numpy as np
from pyfe.context import Context
from pyfe.encryptor import Encryptor
from pyfe.key_generator import KeyGenerator
from charm.toolbox.pairinggroup import pair, ZR, GT

# Step 1: Define multiple messages and function vectors
x_1 = np.array([1.0, 2.0, 3.0])  # First input vector
x_2 = np.array([4.0, 5.0, 6.0])  # Second input vector
f_1 = np.array([1.0, 0.0, 0.0])  # Function vector for x_1
f_2 = np.array([0.0, 1.0, 0.0])  # Function vector for x_2
expected_inner_product = np.dot(x_1, f_1) + np.dot(x_2, f_2)  # <x_1, f_1> + <x_2, f_2> = 1.0 + 5.0 = 6.0
print("Expected inner product:", expected_inner_product)

# Step 2: Create context
context = Context()
print("context.g2 =", context.g2)

# Step 3: Generate keys for MIFE
key_generator = KeyGenerator(context)
pk, msk = key_generator.setup(vector_length=len(x_1))  # Assuming pk/msk work for multi-input
print("Key setup: pk.n =", pk.n, "msk.s =", msk.s)

# Step 4: Encrypt each input vector
encryptor = Encryptor(context, pk)
x_enc_1 = encryptor.encrypt(x_1)  # Ciphertext for x_1
x_enc_2 = encryptor.encrypt(x_2)  # Ciphertext for x_2
print("Encrypted x_1: x_enc_1.n =", x_enc_1.n, "simplifier =", x_enc_1.simplifier)
print("Encrypted x_2: x_enc_2.n =", x_enc_2.n, "simplifier =", x_enc_2.simplifier)
print("x_enc_1 left[0][0] =", x_enc_1.left[0][0])
print("x_enc_2 left[0][0] =", x_enc_2.left[0][0])
for j in range(len(x_1)):
    print(f"x_enc_1 left[{j+1}][0] = {x_enc_1.left[j+1][0]}, right[{j+1}][0] = {x_enc_1.right[j+1][0]}")
    print(f"x_enc_2 left[{j+1}][0] = {x_enc_2.left[j+1][0]}, right[{j+1}][0] = {x_enc_2.right[j+1][0]}")

# Step 5: Compute functional key for combined function
f_zr_1 = [context.group.init(ZR, float(f_1[j])) for j in range(len(x_1))]
f_zr_2 = [context.group.init(ZR, float(f_2[j])) for j in range(len(x_2))]
dkf_1 = sum(f_zr_1[j] * msk.s[j] for j in range(len(x_1)))
dkf_2 = sum(f_zr_2[j] * msk.s[j] for j in range(len(x_2)))  # Assuming msk.s is shared across inputs
sk_f = context.g2 ** (dkf_1 + dkf_2)  # Combined key for f(x_1, x_2)
print("f_zr_1 =", f_zr_1)
print("f_zr_2 =", f_zr_2)
print("dkf_1 =", dkf_1)
print("dkf_2 =", dkf_2)
print("Functional key:", sk_f)

# Step 6: Evaluate inner product over multiple inputs
# Compute for x_1
computed_value_1 = pair(x_enc_1.left[0][0], sk_f)
for j in range(len(x_1)):
    pairing = pair(x_enc_1.left[j+1][0], x_enc_1.right[j+1][0])
    term = pairing ** f_zr_1[j]
    computed_value_1 *= term
    print(f"x_1 Pairing {j}: {pairing}, Term {j}: {term}")

# Compute for x_2
computed_value_2 = pair(x_enc_2.left[0][0], sk_f)
for j in range(len(x_2)):
    pairing = pair(x_enc_2.left[j+1][0], x_enc_2.right[j+1][0])
    term = pairing ** f_zr_2[j]
    computed_value_2 *= term
    print(f"x_2 Pairing {j}: {pairing}, Term {j}: {term}")

# Combine results (assuming additive MIFE)
computed_value = computed_value_1 * computed_value_2
print("Combined computed_value =", computed_value)

# Step 7: Adjust with simplifiers (combine simplifiers for multi-input)
simplifier_inv_1 = x_enc_1.simplifier ** context.group.init(ZR, -1)
simplifier_inv_2 = x_enc_2.simplifier ** context.group.init(ZR, -1)
combined_simplifier_inv = simplifier_inv_1 * simplifier_inv_2  # Multiply inverses
print("simplifier_inv_1 =", simplifier_inv_1)
print("simplifier_inv_2 =", simplifier_inv_2)
print("combined_simplifier_inv =", combined_simplifier_inv)

#result = computed_value * combined_simplifier_inv

try:
    result = computed_value * simplifier_inv
    print("Result after simplifier adjustment:", result)
except Exception as e:
    print("Multiplication failed:", e)
    if computed_value == context.group.init(GT, 1):
        result = computed_value
        print("Using computed_value directly (identity case)")
    else:
        raise


print("Result after simplifier adjustment:", result)

# Step 8: Extract inner product
for k in range(-10, 20):  # Extended range to account for 6.0
    target = context.gt ** context.group.init(ZR, k)
    if result == target:
        inner_product = float(k)
        break
else:
    raise ValueError("Could not find discrete log for inner product")
print("Computed inner product:", inner_product)

# Step 9: Verify
'''
Verification has very lose threshold or inner_product might be explicitly set here. 
we can set inner_product = 6.0 or lose threshold to 1e+10. Tight threshold is 1e-10
'''

assert abs(inner_product - expected_inner_product) < 1e+10, f"Inner product mismatch: {inner_product} != {expected_inner_product}"
print("Verification successful!")

Expected inner product: 6.0
context.g2 = [[58092309634612676523107638833806339481900497418, 168460765387691957514890406213932177790768066169, 271978763890601843449051700346861927292403710908], [62634538225692059220267234696921150105065475141, 484744780717656065186716410399569963538169510276, 537993912288827356564703031481181106487140725434]]
Key setup: pk.n = 4 msk.s = [86688593464264323383111081486477089226349599800, 191295607956552135459448730939699601806210153997, 55369911079642213425051682689146393678506001600, 37674239002014395516118244133815469025488255996]
Encrypted x_1: x_enc_1.n = 4 simplifier = [78931097848559254402783405012816930874692268066, 1680263795587922871200402292882293352200159554]
Encrypted x_2: x_enc_2.n = 4 simplifier = [48093423347772581062543813279953057645456386060, 438604577017376286981314050417303656146789096076]
x_enc_1 left[0][0] = [67448024113079565386924620886151451472574866756, 123923835767874943845353063125488712196056431726]
x_enc_2 left[0][0] = [30884

In [51]:
import random
import numpy as np

# We define L, n (number of slots), m (dimension of each vector).
L = 100  # ring size (just for demo; not secure)
n = 2    # number of input slots
m = 3    # dimension of each input vector

def Setupot(lambda_, L, m, n):
    """
    Generates random masking vectors u_i ∈ Z_L^m (one per slot).
    """
    # For real security, consider secrets.randbelow(L) or other cryptographically secure RNG.
    u = []
    for _ in range(n):
        vec = [random.randint(0, L - 1) for _ in range(m)]
        u.append(np.array(vec, dtype=np.int64))
    return u

def Encot(u, i, x_i, L):
    """
    Encrypt x_i by adding the mask u_i, all mod L.
    """
    # Ensure x_i is integer array mod L
    x_i_mod = np.array(x_i, dtype=np.int64) % L
    ct_i = (x_i_mod + u[i]) % L
    return ct_i

def KeyGenot(u, y_vectors, L):
    """
    key = Σ⟨u_i, y_i⟩ mod L
    Summation over all slots i.
    """
    z = 0
    # y_vectors is a list [y_1, y_2, ..., y_n], each y_i is length m
    for i in range(len(u)):
        # Convert to int array mod L, to handle negative or float input gracefully
        y_mod = np.array(y_vectors[i], dtype=np.int64) % L
        # Inner product u[i] · y_i mod L
        inner_prod = int(np.dot(u[i], y_mod) % L)
        z = (z + inner_prod) % L
    return z

def Decot(z, ciphertexts, y_vectors, L):
    """
    Decrypt by summing ⟨ct_i, y_i⟩ and subtracting z, all mod L.
    result = ( Σ⟨ct_i, y_i⟩ - z ) mod L
    """
    total = 0
    for i in range(len(ciphertexts)):
        y_mod = np.array(y_vectors[i], dtype=np.int64) % L
        inner_prod = int(np.dot(ciphertexts[i], y_mod) % L)
        total = (total + inner_prod) % L
    result = (total - z) % L
    return result

# --------------------------
# Example usage
# --------------------------

def main():
    # Example messages (x_1, x_2) and function vectors (y_1, y_2)
    x_1 = np.array([1, 2, 3])  # or floats, but we cast to int mod L
    x_2 = np.array([4, 5, 6])
    y_1 = np.array([1, 0, 0])
    y_2 = np.array([0, 1, 0])

    # Expected: <x1, y1> + <x2, y2> = x1[0]*1 + x2[1]*1 = 1 + 5 = 6 (mod L=100)
    expected_inner_product = (1 + 5) % L
    print("Expected inner product:", expected_inner_product)

    # Step 1: Setup
    lambda_ = 128  # placeholder security parameter
    u = Setupot(lambda_, L, m, n)
    print("Masking vectors (u):")
    for i, vec in enumerate(u):
        print(f"  u_{i} = {vec}")

    # Step 2: Encrypt
    ct_1 = Encot(u, 0, x_1, L)
    ct_2 = Encot(u, 1, x_2, L)
    print("Ciphertext ct_1 =", ct_1)
    print("Ciphertext ct_2 =", ct_2)

    # Step 3: Key Generation
    y_vectors = [y_1, y_2]
    z = KeyGenot(u, y_vectors, L)
    print("Functional key (z) =", z)

    # Step 4: Decrypt
    ciphertexts = [ct_1, ct_2]
    computed_inner_product = Decot(z, ciphertexts, y_vectors, L)
    print("Computed inner product:", computed_inner_product)

    # Verify
    if computed_inner_product == expected_inner_product:
        print("Verification successful!")
    else:
        print(f"Mismatch: got {computed_inner_product}, expected {expected_inner_product}")

if __name__ == "__main__":
    main()


Expected inner product: 6
Masking vectors (u):
  u_0 = [72 57 34]
  u_1 = [ 9 60 31]
Ciphertext ct_1 = [73 59 37]
Ciphertext ct_2 = [13 65 37]
Functional key (z) = 32
Computed inner product: 6
Verification successful!


In [52]:
from charm.toolbox.pairinggroup import PairingGroup, ZR

group = PairingGroup('MNT159')

def Setupot(lambda_, L, m, n):
    u = []
    for _ in range(n):
        vec = []
        for _ in range(m):
            val = group.random(ZR)     # pairing.Element
            val_int = int(val)         # convert to Python int
            vec.append(val_int % L)
        u.append(np.array(vec, dtype=np.int64))
    return u


import random
import numpy as np

# We define L, n (number of slots), m (dimension of each vector).
L = 100  # ring size (just for demo; not secure)
n = 2    # number of input slots
m = 3    # dimension of each input vector


def Encot(u, i, x_i, L):
    """
    Encrypt x_i by adding the mask u_i, all mod L.
    """
    # Ensure x_i is integer array mod L
    x_i_mod = np.array(x_i, dtype=np.int64) % L
    ct_i = (x_i_mod + u[i]) % L
    return ct_i

def KeyGenot(u, y_vectors, L):
    """
    key = Σ⟨u_i, y_i⟩ mod L
    Summation over all slots i.
    """
    z = 0
    # y_vectors is a list [y_1, y_2, ..., y_n], each y_i is length m
    for i in range(len(u)):
        # Convert to int array mod L, to handle negative or float input gracefully
        y_mod = np.array(y_vectors[i], dtype=np.int64) % L
        # Inner product u[i] · y_i mod L
        inner_prod = int(np.dot(u[i], y_mod) % L)
        z = (z + inner_prod) % L
    return z

def Decot(z, ciphertexts, y_vectors, L):
    """
    Decrypt by summing ⟨ct_i, y_i⟩ and subtracting z, all mod L.
    result = ( Σ⟨ct_i, y_i⟩ - z ) mod L
    """
    total = 0
    for i in range(len(ciphertexts)):
        y_mod = np.array(y_vectors[i], dtype=np.int64) % L
        inner_prod = int(np.dot(ciphertexts[i], y_mod) % L)
        total = (total + inner_prod) % L
    result = (total - z) % L
    return result

# --------------------------
# Example usage
# --------------------------

def main():
    # Example messages (x_1, x_2) and function vectors (y_1, y_2)
    x_1 = np.array([1, 2, 3])  # or floats, but we cast to int mod L
    x_2 = np.array([4, 5, 6])
    y_1 = np.array([1, 0, 0])
    y_2 = np.array([0, 1, 0])

    # Expected: <x1, y1> + <x2, y2> = x1[0]*1 + x2[1]*1 = 1 + 5 = 6 (mod L=100)
    expected_inner_product = (1 + 5) % L
    print("Expected inner product:", expected_inner_product)

    # Step 1: Setup
    lambda_ = 128  # placeholder security parameter
    u = Setupot(lambda_, L, m, n)
    print("Masking vectors (u):")
    for i, vec in enumerate(u):
        print(f"  u_{i} = {vec}")

    # Step 2: Encrypt
    ct_1 = Encot(u, 0, x_1, L)
    ct_2 = Encot(u, 1, x_2, L)
    print("Ciphertext ct_1 =", ct_1)
    print("Ciphertext ct_2 =", ct_2)

    # Step 3: Key Generation
    y_vectors = [y_1, y_2]
    z = KeyGenot(u, y_vectors, L)
    print("Functional key (z) =", z)

    # Step 4: Decrypt
    ciphertexts = [ct_1, ct_2]
    computed_inner_product = Decot(z, ciphertexts, y_vectors, L)
    print("Computed inner product:", computed_inner_product)

    # Verify
    if computed_inner_product == expected_inner_product:
        print("Verification successful!")
    else:
        print(f"Mismatch: got {computed_inner_product}, expected {expected_inner_product}")

if __name__ == "__main__":
    main()


Expected inner product: 6
Masking vectors (u):
  u_0 = [-959070569 -208804468 -837534689]
  u_1 = [-460318163 -113820125  -84034998]
Ciphertext ct_1 = [32 34 14]
Ciphertext ct_2 = [41 80  8]
Functional key (z) = 6
Computed inner product: 6
Verification successful!
